# <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2; text-align: center;">Progress Task 2 (Machine Learning Model Applications and Analysis)</div>

This notebook is a summary of the work done in this second task. We will describe briefly the preprocessing steps, the models used and their performance in the competition and the lessons we have learned.

<div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #0096FF; text-align: center;">FINAL POSITION 478th 
0.8606 score</div>

This score was obtained with Catboost and hyperparameter search with Optuna. 

Boosting algorithms like catboost provided some great performance metrics that let us ascend to position 597. Nevertheless, the amount time and effort put on fine tuning the experiment itself was equally great for the such low results obtained by the team, we expected way more. As a common thought, we all agree that if the dataset had been processed in a better manner, applying outlier and noise reduction techniques, the performance of the model would have been much different (despite the models performing better with the dataset without modifications. But maybe this is caused by low-quality preprocessing).

Here we will show the parameters of the models:

In [8]:
# H1N1 model parameters:
h1n1_best_model = {
  "best_params": {
    "bagging_temperature": 0.06253092805091326,
    "border_count": 245,
    "depth": 5,
    "grow_policy": "Depthwise",
    "iterations": 618,
    "l2_leaf_reg": 0.5332701431047029,
    "learning_rate": 0.047452186432079796,
    "loss_function": "CrossEntropy",
    "random_strength": 1.3505613330374917
  },
  "best_auc_value_training": 0.8741162463651414
}

In [9]:
# Seasonal model parameters:
seasonal_best_model = {
  "best_params": {
    "bagging_temperature": 0.5075887680877953,
    "border_count": 187,
    "depth": 8,
    "grow_policy": "Depthwise",
    "iterations": 683,
    "l2_leaf_reg": 9.962309526689461,
    "learning_rate": 0.0207311303509529,
    "loss_function": "CrossEntropy",
    "random_strength": 2.1124386275626184
  },
  "best_auc_value_training": 0.8686267313659173
}

If we had more time, more iterations of the hyperparameter search could be done to improve the score.

-----------

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Methodology</div>

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">**First attempt.** Random Forest 0.8520</div>

The first thing we did was inspecting the dataset and we realize that it had too many null values. Then, we decided to delete the rows that had these values since the dataset was so large that deleting a few rows wouldn't affect the training.  

**We were wrong.**

When we tried to do the same with the test dataset, we realized that we couldn’t simply ignore the null values and that we needed to handle them properly. For the treatment of nulls in categorical variables, we assigned a new category called 'missing,' and for numerical values, we used -1.
Later we removed some variables that seemed useless employment_industry or hhs_geo and encode both the training and test set using OrdinalEncoder. This is how we obtained the first dataset. After that, we needed to choose a model. 

Based on the results from the first task, we decided to try Random Forest with hyperparameter search, as it was the best among all the classifiers. 

**Result on competition: 0.8520** around 800th place which was around the middle of the leaderboard. Not bad for being the first attempt.


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">**Trying to improve our ranking.** Gradient Boosting 0.8558</div>

To improve our ranking, we decided to follow two approaches: one would explore more Random Forest, and the other would investigate other models.

**1. Other models**

One teammate tried MLPClassifer as an inital approach to Neural Networks while other implemented a complete pipeline using Keras. The third one, tried KNN, but it didn't improve on the Random Forest results. In addition, KNN is influenced by class imbalance and outliers, and we haven't addressed these issues in the dataset.

After some time, we thought about trying other ensemble models. Searching into scikit-learn, we found out about Gradient Boosting models.

**Result on competition: 0.8487 for MLPClassifier**  
**Result on competition: 0.8464 for Neural Networks**, we had some issues with this model.  
**Result on competition: NOT tried for KNN**  
->**Result on competition: 0.8558 for Gradient Boosting**  


**2. Random Forest**

We already had a satisfactory result with Random Forest, but we wondered if it could be improved. Since we decided not to fix any random states, we were able to run the tests multiple times and obtain very different results, which we saved in MLflow.

We tried several approaches:

- Running two separate Random Forest models, one for each target, and optimizing them independently instead of using a multi-output model.
- We also tried filtering the dataset for each target, using only the features relevant to that target. However, this approach performed much worse, as we lost relationships between the features.
- Finally, seeing that all of this was insufficient, we decided to use a "retraining" method on our initial approach. This improved performance, but only slightly.

We will no longer work with Random Forest as we have run out of ideas.

**Result on competition: 0.8538** with the best local Random Forest. 


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">**What's next?: Use AI.** CatBoost 0.8590</div>

We thought we had reached the limit with our approach, but to our surprise, the other teams were making significant improvements. We realized that we could also enhance our ranking by using all the tools available to us: Chat GPT and Claude.

We decided to try some prompt engineering by explaining to Chat GPT the context of our project, the problem we were tackling, and the models we had already tested. We asked for suggestions on other models we could use to improve our ranking score. After "debating" a bit with the chat, we came to the following conclusions:

- We could modify the dataset: This included addressing class imbalance or eliminating correlations.  
- Use basic models: Such as logistic regression or decision trees. However, we completely ruled this out since it wasn't aligned with our goals.  
- **Try more complex models:** Like Gradient Boosting Machines (e.g., XGBoost, LightGBM, CatBoost).  
- Optimize hyperparameters: By using libraries like Optuna or GridSearchCV.

Extracts of the conversations:

1. Is Optuna the same as Random Grid Search in sklearn?

🤖: Not exactly. Although Optuna and Grid/RandomizedSearchCV share the idea of searching for optimal hyperparameters, Optuna has several key differences that make it more flexible and efficient.

2. Explain briefly XGBoost, LightGBM, CatBoost

🤖: All of them are not native sklearn.   
**XGBoost**: Highly optimized for speed and performance but more complex to configure and tune.  
**LightGBM**: Faster than XGBoost in many cases, especially for large and high-dimensional datasets, handles categorical data natively (no encoding required), however can be sensitive to imbalanced data.   
**CatBoost**: Handles categorical features directly, often reduces overfitting automatically but has slower initial training compared to LightGBM.

3. How does Catboost encode categorical variables? Is it worth we manually encode the dataset?  

🤖: No, it's better to use Catboost, let me explain why. Catboost natively uses a technique called 'target-based adaptative encoding' to deal with categorical variables internally. This means that while training, Catboost calculates statistics based on the target variable to encode the features. These statistics are calculated online to prevent leakage information.

*Note*: these are not an exact reproduction of the conversations.

**Therefore we decided to implement CatBoost with Optuna.**

**Result on competition: 0.8590**

------------

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Implementation</div>

❗**NOTE**  
The following section includes the implementation of the models. However, it’s important to note that this notebook was not used to develop the models, and we will not provide a detailed explanation of any of them here. The purpose of this notebook is to act as a summary and to compare the models. For more information, we recommend taking a look at the notebook dedicated to each specific model.

In [2]:
#import libraries
import pandas as pd
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Define a function to create a report of every model.

In [23]:
#Create a table to store the results of the models 
metrics =['Accuracy', 'F1-Macro', 'ROC AUC']
classification_results = pd.DataFrame(columns=metrics)

#Classification Summary Function
def report_classification(y_test, y_pred, model_name, y_pred_proba):
    a = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    roc_auc = roc_auc_score(y_test, y_pred, average="macro")
    classification_results.loc[model_name] = [a, f1, roc_auc]
    
    print("============= METRICS FOR MODEL : ", model_name, " =================")
    print(f"Accuracy: {a}\nF1: {f1}\nRoc_Auc: {roc_auc}")
    print("==================== CONFUSION MATRIX ==========================")
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    # Confusion matrix for h1n1 vaccine
    cm_h1n1 = confusion_matrix(y_test['h1n1_vaccine'], y_pred[:, 0], normalize='all')
    disp_h1n1 = ConfusionMatrixDisplay(confusion_matrix=cm_h1n1, display_labels=['No', 'Yes'])
    disp_h1n1.plot(ax=ax[0], colorbar=False)
    ax[0].set_title('Confusion Matrix for h1n1 Vaccine')

    # Confusion matrix for seasonal vaccine
    cm_seasonal = confusion_matrix(y_test['seasonal_vaccine'], y_pred[:, 1], normalize='all')
    disp_seasonal = ConfusionMatrixDisplay(confusion_matrix=cm_seasonal, display_labels=['No', 'Yes'])
    disp_seasonal.plot(ax=ax[1], colorbar=False)
    ax[1].set_title('Confusion Matrix for Seasonal Vaccine')

    plt.tight_layout()
    plt.show()
    print("==================== ROC AUC Curve ==========================")

    # Get predicted probabilities for positive classes
    y_scores = [p[:, 1] for p in y_pred_proba]

    plt.figure(figsize=(7, 5))
    for i, y_score in enumerate(y_scores):
        fpr, tpr, _ = roc_curve(y_test.iloc[:, i], y_score)
        auc = roc_auc_score(y_test.iloc[:, i], y_score)
        plt.plot(fpr, tpr, label=f'{y_test.columns[i]} (AUC = {auc:.4f})')

    plt.plot([0, 1], [0, 1], 'k--', label="Random")
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

In [ ]:
# import datasets
from mlflow.create_dataset import Dataset

data = Dataset()
X, y = data.with_correlation()
output = data.test

In [4]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 42)

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">Random Forest</div>

In [5]:
from sklearn.ensemble import RandomForestClassifier


model_to_train = MultiOutputClassifier(RandomForestClassifier(warm_start=True), n_jobs=-1)

In [6]:
# initialize search hyperparameters
param_dist_random = {
                'estimator__n_estimators': randint(50, 200),
                'estimator__max_depth': [None, 10, 20, 30],
                'estimator__min_samples_split': randint(2, 11),
                'estimator__min_samples_leaf': randint(1, 5),
                'estimator__criterion': ['gini', 'entropy']
}
random_search = RandomizedSearchCV(estimator=model_to_train, param_distributions=param_dist_random,
                                    n_iter=100, cv=5, n_jobs=-1, verbose=1,
                                    scoring='roc_auc', random_state=42)

In [ ]:
#train the model
# WARNING time: 11 min
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
print('Best parameters found by random search:', random_search.best_params_)

In [ ]:
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)
report_classification(y_test, y_pred, "Random Forest", y_pred_proba)

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">KNN</div>

In [11]:
from sklearn.neighbors import KNeighborsClassifier


model = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=50, weights='uniform'), n_jobs=-1)

model.fit(X_train, y_train)    

y_pred_knn = model.predict(X_test)
y_pred_proba_knn = model.predict_proba(X_test)

In [ ]:
report_classification(y_test, y_pred_knn, "KNN", y_pred_proba_knn)
del y_pred_knn, y_pred_proba_knn

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">MLP Classifier</div>

In [ ]:
from sklearn.neural_network import MLPClassifier

model_to_train = MultiOutputClassifier(MLPClassifier(early_stopping=True, solver='adam', 
                                                     learning_rate='constant', batch_size = 64, 
                                                     max_iter=2000), 
                                       n_jobs=-1)

In [ ]:
# initialize search hyperparameters
param_dist = {
    'estimator__hidden_layer_sizes': [
        (100,), (100, 50), (150,100,50), (100, 50, 25)
    ],
    'estimator__activation': ['tanh', 'relu', 'logistic'],
    'estimator__learning_rate_init': [0.0001, 0.001, 0.01],
    'estimator__alpha': [0.0001, 0.001, 0.01],
}

random_search = RandomizedSearchCV(estimator=model_to_train, 
                                   param_distributions=param_dist, n_iter=100, 
                                   cv=5, n_jobs=-1, verbose=0, scoring = 'roc_auc', random_state=42)

In [ ]:
# train model
# WARNING time: 45 minutes
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
print('Best parameters found by grid search:', random_search.best_params_)

In [ ]:
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)
report_classification(y_test, y_pred_knn, "MLP Classifier", y_pred_proba_knn)

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">Neural Networks</div>

In [ ]:
# further explanations on its notebook

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">Gradient Boosting</div>

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

# initialize model
model_to_train = MultiOutputClassifier(HistGradientBoostingClassifier(early_stopping=True), n_jobs=-1)

# hyperparameters optimization
param_dist_random = {
                'estimator__max_depth': [None, 10, 25, 50, 75, 100],
                'estimator__min_samples_leaf': randint(5, 50),
                'estimator__learning_rate': uniform(0.001, 0.2),
                'estimator__max_iter': randint(50, 500),
                'estimator__l2_regularization': uniform(0.001, 0.01),
                'estimator__max_bins': [50, 100, 200, 255],
    }
random_search = RandomizedSearchCV(estimator=model_to_train, param_distributions=param_dist_random,
                                    n_iter=100, cv=5, n_jobs=-1, verbose=1,
                                    scoring='roc_auc', random_state=42)

In [ ]:
# train model
# WARNING: time 6 min
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
print('Best parameters found by random search:', random_search.best_params_)

In [ ]:
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)
report_classification(y_test, y_pred_knn, "Gradient Boosting", y_pred_proba_knn)

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">LightGBM</div>

In [ ]:
# further explanations on its notebook

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">CatBoost</div>

In [ ]:
# further explanations on its notebook
from catboost import CatBoostClassifier

model = CatBoostClassifier(
        eval_metric='AUC',          
        early_stopping_rounds=5, verbose=0         
)

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Results</div>

In [ ]:
print('=========================== MODELS METRICS =======================================')
plt.figure(figsize=[16,8])
sns.heatmap(classification_results, annot=True, cmap='Blues', fmt='.4f', vmin=0, vmax=1)
plt.show()

Results are saved on MLFlow, organized by experiments and runs. It was easier to compare the performance of each model.

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Challenges faced</div>

1. Communication  
Our main challenge was the lack of communication between the team members. Each one of us was busy during the course and in vacations we were all spending time with our families, which made it difficult to organize our work. We could have improved the repo' structure.

2. Plan in advanced   
Another problem was not having clearly defined what tests we wanted to run. Luckily, one of the teammates insisted on using MLflow, which was incredibly useful to solve this problem. However, in future competitions we should plan in advance which models and tests will be used and share a table to store the results, as all members are supposed to be executing the models.

3. Dataset size  
It was our first time working with a dataset as big as this, with 26700 instances. It was a little difficult to preprocess the dataset because of its huge dimensionality and we had to be very patient because some models took too long to run. This task had a high computational cost both in time and resources.  

4. Competitiveness   
At first, we thought everyone will be using the techniques learnt in class, however, other classmates were ranking higher (maybe using other models), so we had no choice but to step up. At the end, this is a competition but we felt like the rules weren't clear enough. This means that, we can compete with others but among us (the class) it would have been nice to know in advance which was allowed.

5. Optuna   
Despite all of that, we have discovered an extraordinary tool, Optuna. It's very useful for optimizing hyperparameter search, and it allowed us to use the laptop while the training is executing. Also, you can save the execution of the study reload it any time. It would have been very useful to know about this tool from the beginning.